In [1]:
import pickle
import Feature_Extraction
import pandas as pd

haar family: haar
db family: db1, db2, db3, db4, db5, db6, db7, db8, db9, db10, db11, db12, db13, db14, db15, db16, db17, db18, db19, db20, db21, db22, db23, db24, db25, db26, db27, db28, db29, db30, db31, db32, db33, db34, db35, db36, db37, db38
sym family: sym2, sym3, sym4, sym5, sym6, sym7, sym8, sym9, sym10, sym11, sym12, sym13, sym14, sym15, sym16, sym17, sym18, sym19, sym20
coif family: coif1, coif2, coif3, coif4, coif5, coif6, coif7, coif8, coif9, coif10, coif11, coif12, coif13, coif14, coif15, coif16, coif17
bior family: bior1.1, bior1.3, bior1.5, bior2.2, bior2.4, bior2.6, bior2.8, bior3.1, bior3.3, bior3.5, bior3.7, bior3.9, bior4.4, bior5.5, bior6.8
rbio family: rbio1.1, rbio1.3, rbio1.5, rbio2.2, rbio2.4, rbio2.6, rbio2.8, rbio3.1, rbio3.3, rbio3.5, rbio3.7, rbio3.9, rbio4.4, rbio5.5, rbio6.8
dmey family: dmey
gaus family: gaus1, gaus2, gaus3, gaus4, gaus5, gaus6, gaus7, gaus8
mexh family: mexh
morl family: morl
cgau family: cgau1, cgau2, cgau3, cgau4, cgau5, cgau6, cgau7, 

In [2]:
def load_in_participant_data(path):
    with open(path, 'rb') as pickle_file:
        loaded_data = pickle.load(pickle_file)

    NeedToSearch = loaded_data[0]
    CorrectSearch = loaded_data[1]
    IncorrectSearch = loaded_data[2]

    return NeedToSearch, CorrectSearch, IncorrectSearch

In [3]:
import numpy as np
from scipy.signal import butter, filtfilt, hilbert

def bandpass_filter(data, lowcut, highcut, fs, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

# Function to compute oscillatory power measures
def compute_oscillatory_power(eeg_data, sfreq):
    frequency_bands = {
        'theta1': (4, 6),
        'theta2': (6.5, 8),
        'alpha1': (8.5, 10),
        'alpha2': (10.5, 13),
        'beta1': (13.5, 18),
        'beta2': (18.5, 30),
        'gamma1': (30.5, 40),
        'gamma2': (40, 49.5),
    }

    power_data = []

    for band, (lowcut, highcut) in frequency_bands.items():
        # Bandpass filter EEG data
        filtered_data = bandpass_filter(eeg_data, lowcut, highcut, sfreq)

        # Compute the Hilbert transform
        hilbert_transform = hilbert(filtered_data)

        # Compute the instantaneous power
        instantaneous_power = np.abs(hilbert_transform)**2

        # Flatten the power data and append to the feature vector
        power_data.append(instantaneous_power.flatten())

    # Combine power data for all frequency bands into one feature vector
    feature_vector = np.concatenate(power_data)

    return feature_vector

# Example usage:



In [4]:
def create_event_condition_features(event_condition):
    list_of_events = event_condition[0]
    list_of_events_id = event_condition[1]
    Feature_list = []

    for event in list_of_events:
        event_data = event[0]
        features = compute_oscillatory_power(event_data, 400)
        Feature_list.append(features)

    return Feature_list, list_of_events_id


In [6]:

ParticipantList = ["01", "02", "03", "04", "05", "06", "08", "09", "10"]

for participant_number in ParticipantList:

    path = fr"C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Information-Need\Data\DataSegments\participant_0{participant_number}_EEG_data.pkl"

    NeedToSearch, CorrectSearch, IncorrectSearch = load_in_participant_data(path)
    NeedToSearch_features = create_event_condition_features(NeedToSearch)
    CorrectSearch_features = create_event_condition_features(CorrectSearch)
    IncorrectSearch_features = create_event_condition_features(IncorrectSearch)

    Participant_Features = {
        'NeedToSearch': NeedToSearch_features,
        'CorrectSearch': CorrectSearch_features,
        'IncorrectSearch': IncorrectSearch_features
    }

    pickle_file_path = fr'C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Information-Need\Data\Oscillatory_Features_concat\participant_0{participant_number}_Features.pkl'

    # Open the file in binary write mode and use pickle.dump to save the data
    with open(pickle_file_path, 'wb') as pickle_file:
        pickle.dump(Participant_Features, pickle_file)


KeyboardInterrupt: 